1. 총
- roboflow pistols dataset : https://public.roboflow.com/object-detection/pistols/1 (Pistols.v1-resize-416x416.yolov5pytorch.zip)

2. 계단
- Staircase Dataset : https://data.mendeley.com/datasets/7m97gp4yz9/1 (7m97gp4yz9-1.zip)
- Real-time stair detection : https://akshayk07.weebly.com/real-time-stair-detection.html (stairs_dataset.zip)
3. 문
- DoorDetect-Dataset : https://github.com/MiguelARD/DoorDetect-Dataset (DoorDetect-Dataset-master.zip) 1213
-

In [9]:
# coco yolo 데이터셋 각 class별 인덱스 찾는 함수
#
def find_index(object):
    A = ['비행기','사과','가방','바나나','야구 방망이','야구 글로브','곰','침대','벤치','자전거','새','배','책','병','그릇','브로콜리','버스','케익','자동차','당근','고양이','핸드폰','의자','시계','소','컵','식탁','개','도넛','코끼리','소화전','포크','프리스비','기린','헤어 드라이어','핸드백','말','핫도그','키보드','연','칼','노트북','전자레인지','오토바이','마우스','오랜지','오븐','주차요금 징수기','사람','피자','화분','냉장고','리모콘','샌드위치','가위','양','싱크대','스케이트보드','스키','스노우보드','소파','숟가락','공','정지표시','여행가방','서핑보드','테디베어','테니스 라켓','넥타이','토스트기','화장실','칫솔','신호등','기차','트럭','티비','우산','꽃병','와인잔','얼룩말']
    try :
        idx = A.index(object)
        return idx
    except:
        print(object, "없음")
        return -1

In [10]:
nonlabel_object = []

object_list = ['티비',
    '가방',
    '개',
    '냉장고',
    '병',
    '버스',
    '소파',
    '핸드폰',
    '시계',
    '오토바이',
    '와인잔',
    '의자',
    '자동차',
    '책',
    '칼',
    '컵',
    '핸드백',
    '사람',
    '기차',
    '비행기',
    '침대',
    '자전거',
    '우산',
    '고양이',
    '총',
    '계단',
    '문'
    ]
for obj in object_list:
    idx = find_index(obj)
    if idx == -1:
        nonlabel_object.append(obj)

총 없음
계단 없음
문 없음


In [11]:
nonlabel_object

['총', '계단', '문']

In [12]:
from collections import defaultdict
coco_indexTo_newIndex_dict = defaultdict(int)
object_index_dict = defaultdict(int)
for idx in range(len(object_list)):
    object_index_dict[object_list[idx]] = idx
    if find_index(object_list[idx]) != -1:
        coco_indexTo_newIndex_dict[find_index(object_list[idx])] = idx

# 지원하는 object리스트 중 coco dataset에 해당하는 인덱스 모음
coco_index_list = list(coco_indexTo_newIndex_dict.keys())

총 없음
계단 없음
문 없음


In [13]:
print(object_index_dict)
print(coco_indexTo_newIndex_dict)

defaultdict(<class 'int'>, {'티비': 0, '가방': 1, '개': 2, '냉장고': 3, '병': 4, '버스': 5, '소파': 6, '핸드폰': 7, '시계': 8, '오토바이': 9, '와인잔': 10, '의자': 11, '자동차': 12, '책': 13, '칼': 14, '컵': 15, '핸드백': 16, '사람': 17, '기차': 18, '비행기': 19, '침대': 20, '자전거': 21, '우산': 22, '고양이': 23, '총': 24, '계단': 25, '문': 26})
defaultdict(<class 'int'>, {75: 0, 2: 1, 27: 2, 51: 3, 13: 4, 16: 5, 60: 6, 21: 7, 23: 8, 43: 9, 78: 10, 22: 11, 18: 12, 12: 13, 40: 14, 25: 15, 35: 16, 48: 17, 73: 18, 0: 19, 7: 20, 9: 21, 76: 22, 20: 23})


## Train Dataset 전처리

### coco 데이터셋 전처리

In [6]:
import os
# 원래 coco 데이터셋 라벨, 이미지 디렉토리 주소
origianl_train_label_dir = "../dataset/Microsoft COCO.v2-raw.yolov5pytorch/train/labels/"
original_train_image_dir = "../dataset/Microsoft COCO.v2-raw.yolov5pytorch/train/images/"

# 전처리한 coco 데이터셋 라벨, 이미지 디렉토리 주소
change_train_label_dir = "../dataset/change_coco_dataset/train/labels/"
change_train_image_dir = "../dataset/change_coco_dataset/train/images/"

coco_train_label_list = [name for name in os.listdir(origianl_train_label_dir)]

In [7]:
import os, shutil
for label_name in coco_train_label_list:
    # .txt확장자를 갖고 있는 파일 이름을 .jpg확장자를 갖고 있는 파일이름으로 변경
    image_name = os.path.splitext(label_name)[0] +".jpg"

    #원래 라벨 및 이미지 파일 주소 변수
    original_train_label_path = origianl_train_label_dir + label_name
    original_train_image_path = original_train_image_dir + image_name

    #전처리된 라벨 및 이미지 파일 주소 변수
    change_train_label_path = change_train_label_dir + label_name
    change_train_image_path = change_train_image_dir + image_name

    with open(original_train_label_path,'r') as file:
        # 파일에 쓸 문자열 저장 변수
        new_string = []
        # 지원하는 object가 있는 파일 인지 확인 하는 flag
        exist_flag = False

        # 지원하는 object class에 맞게 전처리
        for line in file:
            data = line.split()
            # 지원하는 object가 있으면 new_string 추가
            if int(data[0]) in coco_index_list:
                # coco dataset의 index를 지원하는 object의 순서에 맞게 index 변경
                data[0] = str(coco_indexTo_newIndex_dict[int(data[0])])
                new_string.append(" ".join(data))
                # 지원하는 object가 라벨파일에 있으면 flag를 true로 변경
                exist_flag = True
        # 지원하는 object가 있으면 전처리된 라벨 데이터 파일에 쓰고 해당하는 이미지 복사
        if exist_flag == True:
            with open(change_train_label_path,'w') as change_file:
                change_file.write("\n".join(new_string))
            shutil.copy(original_train_image_path, change_train_image_path)

### 문 데이터 전처리

In [16]:
# 일부분 오류 데이터 삭제했음

import os
# 원래 coco 데이터셋 라벨, 이미지 디렉토리 주소
door1_origianl_train_label_dir = "../dataset/DoorDetect-Dataset-master/labels/"
door1_original_train_image_dir = "../dataset/DoorDetect-Dataset-master/images/"

# 전처리한 coco 데이터셋 라벨, 이미지 디렉토리 주소
door1_change_train_label_dir = "../dataset/change_door_dataset/train/labels/"
door1_change_train_image_dir = "../dataset/change_door_dataset/train/images/"

door1_train_label_list = [name for name in os.listdir(door1_origianl_train_label_dir)]

In [17]:
import os, shutil
for label_name in door1_train_label_list:
    if len(label_name) >=110:
        continue

    # .txt확장자를 갖고 있는 파일 이름을 .jpg확장자를 갖고 있는 파일이름으로 변경
    image_name = os.path.splitext(label_name)[0] +".jpg"

    #원래 라벨 및 이미지 파일 주소 변수
    original_train_label_path = door1_origianl_train_label_dir + label_name
    original_train_image_path = door1_original_train_image_dir + image_name

    #전처리된 라벨 및 이미지 파일 주소 변수
    change_train_label_path = door1_change_train_label_dir + label_name
    change_train_image_path = door1_change_train_image_dir + image_name

    with open(original_train_label_path,'r') as file:
        # 파일에 쓸 문자열 저장 변수
        new_string = []
        # 지원하는 object가 있는 파일 인지 확인 하는 flag
        exist_flag = False

        # 지원하는 object class에 맞게 전처리
        for line in file:
            data = line.split()
            # 문에 해당하는 id로 class 인덱스
            if data[0] == "0" or data[0] == "2" or data[0] == "3":
                data[0] = str(26)
                new_string.append(" ".join(data))
                # 지원하는 object가 라벨파일에 있으면 flag를 true로 변경
                exist_flag = True
        # 지원하는 object가 있으면 전처리된 라벨 데이터 파일에 쓰고 해당하는 이미지 복사
        if exist_flag == True:
            with open(change_train_label_path,'w') as change_file:
                change_file.write("\n".join(new_string))
            shutil.copy(original_train_image_path, change_train_image_path)

In [37]:
import numpy as np
from sklearn.model_selection import train_test_split
import os
# 이미지,라벨 데이터 이름 정렬
image_list = sorted([_ for _ in os.listdir(door1_change_train_image_dir)])
label_list = sorted([_ for _ in os.listdir(door1_change_train_label_dir)])
# train, test로 나누기
train_names, valid_names = train_test_split(image_list, test_size=0.2, random_state=42, shuffle=True)
trainLabels_names, validLabels_names = train_test_split(label_list, test_size=0.2, random_state=42, shuffle=True)

In [38]:
# train 이미지랑 라벨 같은지 확인
for image, label in zip(train_names,trainLabels_names):
  if image[:-4] != label[:-4]:
    print(image,label)

In [39]:
# train 이미지랑 라벨 같은지 확인
for image, label in zip(valid_names,validLabels_names):
  if image[:-4] != label[:-4]:
    print(image,label)

In [40]:
import shutil
for image, label in zip(train_names,trainLabels_names):
  shutil.move(door1_change_train_image_dir+image,"../dataset/change_door_dataset/valid/images/")
  shutil.move(door1_change_train_label_dir+label,"../dataset/change_door_dataset/valid/labels/")

### 총 데이터셋 전처리

In [20]:
import os
# 원래 coco 데이터셋 라벨, 이미지 디렉토리 주소
gun1_origianl_train_label_dir = "../dataset/Pistols.v1-resize-416x416.yolov5pytorch/export/labels/"
gun1_original_train_image_dir = "../dataset/Pistols.v1-resize-416x416.yolov5pytorch/export/images/"

# 전처리한 coco 데이터셋 라벨, 이미지 디렉토리 주소
gun1_change_train_label_dir = "../dataset/change_gun_dataset/train/labels/"
gun1_change_train_image_dir = "../dataset/change_gun_dataset/train/images/"

gun1_train_label_list = [name for name in os.listdir(gun1_origianl_train_label_dir)]

In [23]:
import os, shutil
for label_name in gun1_train_label_list:

    # .txt확장자를 갖고 있는 파일 이름을 .jpg확장자를 갖고 있는 파일이름으로 변경
    image_name = os.path.splitext(label_name)[0] +".jpg"

    #원래 라벨 및 이미지 파일 주소 변수
    original_train_label_path = gun1_origianl_train_label_dir + label_name
    original_train_image_path = gun1_original_train_image_dir + image_name

    #전처리된 라벨 및 이미지 파일 주소 변수
    change_train_label_path = gun1_change_train_label_dir + label_name
    change_train_image_path = gun1_change_train_image_dir + image_name

    with open(original_train_label_path,'r') as file:
        # 파일에 쓸 문자열 저장 변수
        new_string = []
        # 지원하는 object가 있는 파일 인지 확인 하는 flag
        exist_flag = False

        # 지원하는 object class에 맞게 전처리
        for line in file:
            data = line.split()
            # 문에 해당하는 id로 class 인덱스
            if data[0] == "0":
                data[0] = str(24)
                new_string.append(" ".join(data))
                # 지원하는 object가 라벨파일에 있으면 flag를 true로 변경
                exist_flag = True
        # 지원하는 object가 있으면 전처리된 라벨 데이터 파일에 쓰고 해당하는 이미지 복사
        if exist_flag == True:
            with open(change_train_label_path,'w') as change_file:
                change_file.write("\n".join(new_string))
            shutil.copy(original_train_image_path, change_train_image_path)


In [41]:
import numpy as np
from sklearn.model_selection import train_test_split
import os

# 이미지,라벨 데이터 이름 정렬
image_list = sorted([_ for _ in os.listdir(gun1_change_train_image_dir)])
label_list = sorted([_ for _ in os.listdir(gun1_change_train_label_dir)])
# train, test로 나누기
train_names, valid_names = train_test_split(image_list, test_size=0.2, random_state=42, shuffle=True)
trainLabels_names, validLabels_names = train_test_split(label_list, test_size=0.2, random_state=42, shuffle=True)

In [42]:
# train 이미지랑 라벨 같은지 확인
for image, label in zip(train_names,trainLabels_names):
  if image[:-4] != label[:-4]:
    print(image,label)

In [43]:
# train 이미지랑 라벨 같은지 확인
for image, label in zip(valid_names,validLabels_names):
  if image[:-4] != label[:-4]:
    print(image,label)

In [44]:
import shutil
for image, label in zip(train_names,trainLabels_names):
  shutil.move(gun1_change_train_image_dir+image,"../dataset/change_gun_dataset/valid/images/")
  shutil.move(gun1_change_train_label_dir+label,"../dataset/change_gun_dataset/valid/labels/")

### 계단 데이터셋 전처리1

In [29]:
import os
# 원래 coco 데이터셋 라벨, 이미지 디렉토리 주소
stair1_origianl_train_label_dir = "../dataset/stairs_dataset/train/"
stair1_original_train_image_dir = "../dataset/stairs_dataset/train/"

# 전처리한 coco 데이터셋 라벨, 이미지 디렉토리 주소
stair1_change_train_label_dir = "../dataset/change_stair1_dataset/train/labels/"
stair1_change_train_image_dir = "../dataset/change_stair1_dataset/train/images/"

stair1_train_label_list = [name for name in os.listdir(stair1_origianl_train_label_dir) if name.split(".")[-1] == "txt"]

In [30]:
import os, shutil
for label_name in stair1_train_label_list:

    # .txt확장자를 갖고 있는 파일 이름을 .jpg확장자를 갖고 있는 파일이름으로 변경
    image_name = os.path.splitext(label_name)[0] +".jpg"

    #원래 라벨 및 이미지 파일 주소 변수
    original_train_label_path = stair1_origianl_train_label_dir + label_name
    original_train_image_path = stair1_original_train_image_dir + image_name

    #전처리된 라벨 및 이미지 파일 주소 변수
    change_train_label_path = stair1_change_train_label_dir + label_name
    change_train_image_path = stair1_change_train_image_dir + image_name

    with open(original_train_label_path,'r') as file:
        # 파일에 쓸 문자열 저장 변수
        new_string = []
        # 지원하는 object가 있는 파일 인지 확인 하는 flag
        exist_flag = False

        # 지원하는 object class에 맞게 전처리
        for line in file:
            data = line.split()
            # 문에 해당하는 id로 class 인덱스
            if data[0] == "0":
                data[0] = str(25)
                new_string.append(" ".join(data))
                # 지원하는 object가 라벨파일에 있으면 flag를 true로 변경
                exist_flag = True
        # 지원하는 object가 있으면 전처리된 라벨 데이터 파일에 쓰고 해당하는 이미지 복사
        if exist_flag == True:
            with open(change_train_label_path,'w') as change_file:
                change_file.write("\n".join(new_string))
            shutil.copy(original_train_image_path, change_train_image_path)


In [45]:
import numpy as np
from sklearn.model_selection import train_test_split
import os

# 이미지,라벨 데이터 이름 정렬
image_list = sorted([_ for _ in os.listdir(stair1_change_train_image_dir)])
label_list = sorted([_ for _ in os.listdir(stair1_change_train_label_dir)])
# train, test로 나누기
train_names, valid_names = train_test_split(image_list, test_size=0.2, random_state=42, shuffle=True)
trainLabels_names, validLabels_names = train_test_split(label_list, test_size=0.2, random_state=42, shuffle=True)

In [46]:
# train 이미지랑 라벨 같은지 확인
for image, label in zip(train_names,trainLabels_names):
  if image[:-4] != label[:-4]:
    print(image,label)

In [47]:
# train 이미지랑 라벨 같은지 확인
for image, label in zip(valid_names,validLabels_names):
  if image[:-4] != label[:-4]:
    print(image,label)

In [48]:
import shutil
for image, label in zip(train_names,trainLabels_names):
  shutil.move(stair1_change_train_image_dir+image,"../dataset/change_stair1_dataset/valid/images/")
  shutil.move(stair1_change_train_label_dir+label,"../dataset/change_stair1_dataset/valid/labels/")

### 계단 데이터셋 전처리2

## Valid 데이터셋 전처리

In [14]:
import os
# 원래 coco 데이터셋 라벨, 이미지 디렉토리 주소
origianl_valid_label_dir = "../dataset/Microsoft COCO.v2-raw.yolov5pytorch/valid/labels/"
original_valid_image_dir = "../dataset/Microsoft COCO.v2-raw.yolov5pytorch/valid/images/"

# 전처리한 coco 데이터셋 라벨, 이미지 디렉토리 주소
change_valid_label_dir = "../dataset/change_coco_dataset/valid/labels/"
change_valid_image_dir = "../dataset/change_coco_dataset/valid/images/"

coco_valid_label_list = [path for path in os.listdir(origianl_valid_label_dir)]

In [15]:
import os, shutil
for label_name in coco_valid_label_list:
    # .txt확장자를 갖고 있는 파일 이름을 .jpg확장자를 갖고 있는 파일이름으로 변경
    image_name = os.path.splitext(label_name)[0] +".jpg"

    #원래 라벨 및 이미지 파일 주소 변수
    original_valid_label_path = origianl_valid_label_dir + label_name
    original_valid_image_path = original_valid_image_dir + image_name

    #전처리된 라벨 및 이미지 파일 주소 변수
    change_valid_label_path = change_valid_label_dir + label_name
    change_valid_image_path = change_valid_image_dir + image_name

    with open(original_valid_label_path,'r') as file:
        # 파일에 쓸 문자열 저장 변수
        new_string = []
        # 지원하는 object가 있는 파일 인지 확인 하는 flag
        exist_flag = False

        # 지원하는 object class에 맞게 전처리
        for line in file:
            data = line.split()
            # 지원하는 object가 있으면 new_string 추가
            if int(data[0]) in coco_index_list:
                # coco dataset의 index를 지원하는 object의 순서에 맞게 index 변경
                data[0] = str(coco_indexTo_newIndex_dict[int(data[0])])
                new_string.append(" ".join(data))
                # 지원하는 object가 라벨파일에 있으면 flag를 true로 변경
                exist_flag = True
        # 지원하는 object가 있으면 전처리된 라벨 데이터 파일에 쓰고 해당하는 이미지 복사
        if exist_flag == True:
            with open(change_valid_label_path,'w') as change_file:
                change_file.write("\n".join(new_string))
            shutil.copy(original_valid_image_path, change_valid_image_path)

In [ ]:
# Yolov5 github 레포지토리 clone
!git clone https://github.com/ultralytics/yolov5.git

In [10]:
%cd yolov5-master

C:\Users\tlsgh\PycharmProjects\ObjectDetect\code\yolov5-master


In [ ]:
# 필요한 모듈 설치
!pip install -U -r requirements.txt


In [ ]:
!conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

In [19]:
import torch
torch.cuda.is_available()

False

In [17]:
# 훈련시키기
!python ./train.py --data "./data.yaml" --img 128 --epochs 100

^C
